Install NCBI Datasets command line tool and API library.

In [ ]:
!curl -o datasets 'https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/linux-amd64/datasets' && chmod +x datasets

In [ ]:
!pip3 install ncbi-datasets-pylib

In [ ]:
!pip3 install biopython
!pip3 install pandas

In [ ]:
import sys
import zipfile
import pandas as pd
import yaml
from pprint import pprint
from datetime import datetime
from collections import defaultdict, Counter
from IPython.display import display
from Bio.SeqFeature import FeatureLocation, CompoundLocation

try:
    import ncbi.datasets
except ImportError:
    print('ncbi.datasets module not found. To install, run `pip install ncbi-datasets-pylib`.')

Documentation of some useful APIs.

In [119]:
help(Bio.SeqFeature.FeatureLocation)
help(Bio.SeqFeature.CompoundLocation)

Help on class FeatureLocation in module Bio.SeqFeature:

class FeatureLocation(builtins.object)
 |  FeatureLocation(start, end, strand=None, ref=None, ref_db=None)
 |  
 |  Specify the location of a feature along a sequence.
 |  
 |  The FeatureLocation is used for simple continuous features, which can
 |  be described as running from a start position to and end position
 |  (optionally with a strand and reference information).  More complex
 |  locations made up from several non-continuous parts (e.g. a coding
 |  sequence made up of several exons) are described using a SeqFeature
 |  with a CompoundLocation.
 |  
 |  Note that the start and end location numbering follow Python's scheme,
 |  thus a GenBank entry of 123..150 (one based counting) becomes a location
 |  of [122:150] (zero based counting).
 |  
 |  >>> from Bio.SeqFeature import FeatureLocation
 |  >>> f = FeatureLocation(122, 150)
 |  >>> print(f)
 |  [122:150]
 |  >>> print(f.start)
 |  122
 |  >>> print(f.end)
 |  150


NOTE: Failed attempt to use NCBI Datasets API.

In [ ]:
with ncbi.datasets.ApiClient() as api_client:
    # Create an instance of the API class
    api_instance = ncbi.datasets.DownloadApi(api_client)
    gene_ids = [7113]
    include_sequence_type = ['SEQ_TYPE_GENE', 'SEQ_TYPE_RNA', 'SEQ_TYPE_PROTEIN']
    filename = 'sars-cov2-genes'
    
    try:
        api_response = api_instance.download_gene_package(gene_ids,
                            filename=filename)
    except ncbi.datasets.ApiException as e:
        print("Exception when calling DownloadApi->download_gene_package: %s\n" % e)

ALTERNATIVE: Use NCBI Datasets command line tool.

In [ ]:
!./datasets download gene 59272

In [ ]:
! rm -rf sars-cov2-gene-data
! unzip -d sars-cov2-gene-data ncbi_dataset.zip

In [116]:
with open('sars-cov2-gene-data/ncbi_dataset/data/data_report.yaml') as yaml_file:
    gene_data = yaml.load(yaml_file, Loader=yaml.SafeLoader)
pprint(gene_data)

{'genes': [{'chromosomes': ['X'],
            'commonName': 'human',
            'description': 'angiotensin I converting enzyme 2',
            'ensemblGeneIds': ['ENSG00000130234'],
            'geneId': '59272',
            'genomicRanges': [{'accessionVersion': 'NC_000023.11',
                               'range': [{'begin': '15561033',
                                          'end': '15602158',
                                          'orientation': 'minus'}]}],
            'nomenclatureAuthority': {'authority': 'HGNC',
                                      'identifier': 'HGNC:13557'},
            'omimIds': ['300335'],
            'swissProtAccessions': ['Q9BYF1.2'],
            'symbol': 'ACE2',
            'synonyms': ['ACEH'],
            'taxId': '9606',
            'taxname': 'Homo sapiens',
            'transcripts': [{'accessionVersion': 'NM_021804.3',
                             'cds': {'accessionVersion': 'NM_021804.3',
                                     'range': 

In [ ]:
def get_strand(orientation):
    """Convert NCBI Dataset data report orientation to BioPython strand.
    """
    return {'minus':-1, 'plus':+1, None:None}[orientation]

def make_simple_location(interval, ref, default_strand=None):
    """Convert NCBI Dataset data report range element to BioPython
    FeatureLocation.
    
    Note: Data report is 1-based, BioPython is 0-based.
    """
    strand = get_orientation(interval.get('orientation', None))
    if strand is None:
        strand = default_strand
    return FeatureLocation(int(interval['begin'])-1,
                           int(interval['end'])-1,
                           strand,
                           ref)

def make_location(location_data, default_strand=None):
    """Convert NCBI Dataset data report location to BioPython
    FeatureLocation or CompoundLocation.
    
    Note: Data report is 1-based, BioPython is 0-based.
    """

    ref = location_data['accessionVersion']
    if len(location_data['range']) == 1:
        return make_simple_location(location_data['range'][0],
                                    ref,
                                    default_strand)

    loc = []
    for interval in location_data['range']:
        loc.append(make_simple_location(interval, ref, default_strand))
    return CompoundLocation(loc)

def clip_location(position):
    """Clip a postion for a BioPython FeatureLocation to the reference.
    
    WARNING: Use of reference length not implemented yet.
    """
    return max(0, position)

def gene_to_upstream(gene, upstream_length = 2000):
    """Given a gene structure from an NCBI Dataset gene data report,
    return a list of rows with upstream range and the set of
    transcripts sharing that upstream range.
    
    Example: [[FeatureLocation(...), 'NM_123', 'NM_456']]
    """
    upstream_collection = dict()
    # Need strand from gene location, since it's not included in exon locations.
    gene_location = make_location(gene['genomicRanges'][0])

    for transcript in gene['transcripts']:
        transcript_accession = transcript['accessionVersion']
        transcript_location = make_location(transcript['exons'], gene_location.strand)

        if transcript_location.strand < 0:
            upstream = FeatureLocation(max(transcript_location) + 1,
                                       max(transcript_location) + upstream_length,
                                       transcript_location.strand,
                                       gene_location.ref)
        else:
            upstream = FeatureLocation(clip_location(min(transcript_location) - upstream_length),
                                       clip_location(min(transcript_location) - 1),
                                       transcript_location.strand,
                                       gene_location.ref)

        # BioSeq FeatureLocation not hashable.
        upstream_key = (min(upstream), max(upstream), upstream.strand, upstream.ref)
        if upstream_key in upstream_collection:
            upstream_collection[upstream_key][1].append(transcript_accession)
        else:
            upstream_collection[upstream_key] = [upstream, transcript_accession]
    return list(upstream_collection.values())

In [118]:
for gene in gene_data['genes']:
    print(f'Report for gene {gene["symbol"]}')
    upstream_collection = gene_to_upstream(gene)
    print(upstream_collection)

Report for gene ACE2
[[FeatureLocation(ExactPosition(15602157), ExactPosition(15604156), strand=-1, ref='NC_000023.11'), 'NM_021804.3'], [FeatureLocation(ExactPosition(15600959), ExactPosition(15602958), strand=-1, ref='NC_000023.11'), 'NM_001371415.1']]
